In [ ]:
!pip install -q fastapi uvicorn pyngrok nest_asyncio transformers torch accelerate tiktoken streamlit python-dotenv
!pip install -q huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 106.3 MB/s eta 0:00:00


In [ ]:
# colab_server.py

import os
import threading
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List, Optional
import nest_asyncio
from pyngrok import ngrok
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# 1. Setup Model
MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
# Note: You need to accept terms on HF and have a token.
# Add your HF token below or log in via CLI
HF_TOKEN = "hf_...your_token_here..."

print("Loading Model... (This takes a few minutes)")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    dtype=torch.float16,
    device_map="auto",
    token=HF_TOKEN
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

# 2. Setup FastAPI
app = FastAPI()

class Message(BaseModel):
    role: str
    content: str

class ChatCompletionRequest(BaseModel):
    model: str
    messages: List[Message]
    temperature: Optional[float] = 0.7
    max_tokens: Optional[int] = 1024

@app.post("/v1/chat/completions")
async def chat_completions(request: ChatCompletionRequest):
    try:
        # Convert messages to Llama-3 prompt format
        prompt = tokenizer.apply_chat_template(
            [m.dict() for m in request.messages],
            tokenize=False,
            add_generation_prompt=True
        )

        outputs = pipe(
            prompt,
            max_new_tokens=request.max_tokens,
            temperature=request.temperature,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )

        generated_text = outputs[0]["generated_text"]
        # Extract only the response (remove the prompt)
        response_content = generated_text[len(prompt):].strip()

        return {
            "id": "chatcmpl-123",
            "object": "chat.completion",
            "created": 1677652288,
            "choices": [{
                "index": 0,
                "message": {
                    "role": "assistant",
                    "content": response_content,
                },
                "finish_reason": "stop"
            }],
            "usage": {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0}
        }
    except Exception as e:
        print(f"Error: {e}")
        raise HTTPException(status_code=500, detail=str(e))

# 3. Start Server with ngrok
def start_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

if __name__ == "__main__":
    # Add your ngrok authtoken
    ngrok.set_auth_token("your_ngrok_auth_token_here")

    public_url = ngrok.connect(8000).public_url
    print(f"\n API BASE URL: {public_url}/v1")

    nest_asyncio.apply()
    thread = threading.Thread(target=start_server, daemon=True)
    thread.start()

    # Keep main thread alive
    import time
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Stopping server...")

Loading Model... (This takes a few minutes)


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...


 API BASE URL: https://cedric-unstony-fulsomely.ngrok-free.dev/v1


INFO:     Started server process [2311]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
/tmp/ipython-input-263321320.py:57: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  [m.dict() for m in request.messages],


INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b1be:f39c:8074:2d0e:0 - "POST /v1/chat/completions HTTP/1.1" 200 OK
INFO:     2402:800:6315:9626:b